# **Initial Exploration**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

Section 1 content

In [ ]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/airruns.csv",index_col=0)
df.head()

In [ ]:
df.info()

In [ ]:
# Drop atmospheric data, name, location and county, race no
df = pd.read_csv(f"inputs/datasets/raw/airruns.csv",index_col=0).drop(['co','no','no2','o3','so2','pm2_5','pm10','nh3', 'name','race_num','location', 'county'], axis=1)
df.info()

In [ ]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, title='Cheeky Explore', minimal=True)
pandas_report.to_notebook_iframe()

___


# Target Encoding / Mean Encoding

In [ ]:
from feature_engine.encoding import MeanEncoder
# Create an instance of the MeanEncoder
encoder = MeanEncoder(variables=['parkrun'])
# isinstance(df['parkrun'],pd.DataFrame)
# Fit the encoder on the 'parkrun' column
encoder.fit(df, df['time'])
# Transform the 'parkrun' column
transformed_df = encoder.transform(df)
transformed_df = pd.concat([df['time'], transformed_df], axis=1)
print(transformed_df.shape)
transformed_df.head()

___


## Correlation Study

In [ ]:
corr_spearman = processed_df.corr(method='spearman', numeric_only=False)['time'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
